$\newcommand{\To}{\Rightarrow}$
$\newcommand{\Suc}{\operatorname{Suc}\,}$
$\newcommand{\zero}{\mathrm{zero}}$
$\newcommand{\one}{\mathrm{one}}$
$\newcommand{\bitz}{\operatorname{bit0}\,}$
$\newcommand{\bito}{\operatorname{bit1}\,}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import NatType
from kernel.term import Var, Nat, Inst, Eq, Lambda, Binary
from kernel import theory
from kernel.proofterm import refl, ProofTerm
from logic import basic
from logic.conv import top_conv, rewr_conv, every_conv, try_conv, arg_conv, repeat_conv, Conv
from logic.logic import apply_theorem
from data.nat import Suc, zero
from syntax.settings import settings

basic.load_theory('nat')
settings.unicode = True

## Peano arithmetic

In higher-order logic, types like natural numbers are defined as an inductive datatype, following the construction of natural numbers using Peano arithmetic. This construction specifies that the natural numbers can be built through two constructors: $\zero :: nat$ and $\Suc :: nat \To nat$ (here $\hbox{Suc}$ is short for successor). This is usually written as:

$$\begin{align*}
\hbox{datatype }nat = \zero\ |\ \Suc nat
\end{align*}$$

This means terms like $\zero, \Suc\zero, \Suc(\Suc \zero)$, etc are natural numbers. Moreover, these are the *only* natural numbers. It is clear that the numbers above correspond to $0, 1, 2$, etc.

The fact that these are the only natural numbers is reflected in the following theorem, called *principle of mathematical induction*:

In [3]:
theory.print_theorem('nat_induct')

nat_induct: ⊢ P 0 ⟶ (∀n. P n ⟶ P (Suc n)) ⟶ P x


The theorem says that, in order to prove any property $P$ holds for all natural numbers, it suffices to show that $P$ holds for $0$, and that if $P$ holds for $n$ then it holds for $\Suc n$ (that is, $n+1$). The reason we insist on using $\Suc n$ instead of $n+1$ is that so far, we are assuming that we have just defined natural numbers, and have not defined addition or proved any of its properties. This is what we will do next.

Addition on natural numbers is also defined by induction, more precisely, we define $m+n$ by induction on the first argument $m$. The definition consists of two equations, displayed as follows:

$$
\begin{align*}
& \mathbf{fun} \hbox{ plus} :: nat \To nat \To nat\ \mathbf{where} \\
& \quad 0 + n = n \\
& \quad \hbox{Suc } m + n = \hbox{Suc } (m + n)
\end{align*}
$$

The two equalities are given names `nat_plus_def_1` and `nat_plus_def_2` in holpy:

In [4]:
theory.print_theorem('nat_plus_def_1', 'nat_plus_def_2')

nat_plus_def_1: ⊢ 0 + n = n
nat_plus_def_2: ⊢ Suc m + n = Suc (m + n)


If we interpret $\Suc n$ as $n+1$, it is clear that these equations are true. Morever, these two equations suffice to compute the sum of any two natural numbers expressed using $\Suc$ and $\zero$. For example, the following computes $2+1=3$:

$$ \Suc(\Suc\zero) + \Suc\zero = \Suc(\Suc\zero + \Suc\zero) = \Suc(\Suc(\zero + \Suc\zero)) = \Suc(\Suc(\Suc\zero)) $$

Please check that the above calculation consists of two applications of `nat_plus_def_2`, followed by one application of `nat_plus_def_1`.

We can realize this in Python as follows:

In [5]:
norm_plus_cv = top_conv('nat_plus_def_1', 'nat_plus_def_2')

t = Suc(Suc(zero)) + Suc(zero)
print('t:', t)
print('eq:', refl(t).on_rhs(norm_plus_cv))

t: Suc (Suc 0) + Suc 0
eq: ProofTerm(⊢ Suc (Suc 0) + Suc 0 = Suc (Suc (Suc 0)))


Multiplication on natural numbers is defined in a similar way using induction:

$$
\begin{align*}
& \mathbf{fun} \hbox{ times} :: nat \To nat \To nat\ \mathbf{where} \\
& \quad 0 * n = 0 \\
& \quad \hbox{Suc } m * n = n + m * n
\end{align*}
$$

This is recorded as the following theorems:

In [6]:
theory.print_theorem('nat_times_def_1', 'nat_times_def_2')

nat_times_def_1: ⊢ 0 * n = 0
nat_times_def_2: ⊢ Suc m * n = n + m * n


We can compute with multiplication as follows:

In [7]:
norm_times_cv = top_conv('nat_times_def_1', 'nat_times_def_2', norm_plus_cv)

t = Suc(Suc(zero)) * Suc(Suc(Suc(zero)))
print('t:', t)
print(refl(t).on_rhs(norm_times_cv))

t: Suc (Suc 0) * Suc (Suc (Suc 0))
ProofTerm(⊢ Suc (Suc 0) * Suc (Suc (Suc 0)) = Suc (Suc (Suc (Suc (Suc (Suc 0))))))


## Proofs using induction

We now give some examples of proofs using induction. First, note that while we know $0 + n = n$ directly from definition, the fact that $n + 0 = n$ still need to be proved. The proof is by induction on $n$. By induction, we need to prove $0+0=0$ and $\forall n.\,n+0=n \to \Suc n+0=\Suc n$.

#### Example:

Prove $n + 0 = n$.

#### Solution:

0. $\vdash 0 + 0 = 0$ by rewriting with nat_plus_def_1.
1. $n + 0 = n \vdash n + 0 = n$ by assume $n + 0 = n$.
2. $\vdash \Suc(n) + 0 = \Suc(n + 0)$ by rewriting with nat_plus_def_2.
3. $n + 0 = n \vdash \Suc(n) + 0 = \Suc(n)$ by rewriting 2 with 1.
4. $\vdash n + 0 = n \to \Suc(n) + 0 = \Suc(n)$ by implies_intr $n + 0 = n$ from 3.
5. $\vdash \forall n.\,n + 0 = n \to \Suc(n) + 0 = \Suc(n)$ by forall_intr $n$ from 4.
6. $n + 0 = n$ by apply_theorem nat_induct from 0, 5.

The proof can be realized in Python as follows:

In [8]:
n = Var('n', NatType)

pt0 = refl(Nat(0) + 0).on_rhs(rewr_conv('nat_plus_def_1'))
pt1 = ProofTerm.assume(Eq(n + 0, n))
pt2 = refl(Suc(n) + 0).on_rhs(rewr_conv('nat_plus_def_2'))
pt3 = pt2.on_rhs(arg_conv(rewr_conv(pt1)))
pt4 = pt3.implies_intr(Eq(n + 0, n))
pt5 = pt4.forall_intr(n)
pt6 = apply_theorem('nat_induct', pt0, pt5, inst=Inst(P=Lambda(n, Eq(n + 0, n)), x=n))
print(pt6)

ProofTerm(⊢ n + 0 = n)


The exported proof is as follows. It is a bit longer than the proof written above because of extra steps inserted by conversions.

In [9]:
print(pt6.export())

0: ⊢ 0 + ?n = ?n by theorem nat_plus_def_1
1: ⊢ (0::nat) + 0 = 0 by substitution {n: (0::nat)} from 0
2: ⊢ Suc ?m + ?n = Suc (?m + ?n) by theorem nat_plus_def_2
3: ⊢ Suc n + 0 = Suc (n + 0) by substitution {m: n, n: (0::nat)} from 2
4: ⊢ Suc = Suc by reflexive Suc
5: n + 0 = n ⊢ n + 0 = n by assume n + 0 = n
6: n + 0 = n ⊢ Suc (n + 0) = Suc n by combination from 4, 5
7: n + 0 = n ⊢ Suc n + 0 = Suc n by transitive from 3, 6
8: ⊢ n + 0 = n ⟶ Suc n + 0 = Suc n by implies_intr n + 0 = n from 7
9: ⊢ ∀n. n + 0 = n ⟶ Suc n + 0 = Suc n by forall_intr n from 8
10: ⊢ n + 0 = n by apply_theorem_for nat_induct, {P: λn::nat. n + 0 = n, x: n} from 1, 9


We do a second example, this time concerning multiplication.

#### Example:

Prove $n * 0 = 0$.

#### Solution:

0. $\vdash 0 * 0 = 0$ by rewriting with nat_times_def_1.
1. $n * 0 = 0 \vdash n * 0 = 0$ by assume $n * 0 = 0$.
2. $\vdash \Suc(n) * 0 = 0 + n * 0$ by rewriting with nat_times_def_2.
3. $\vdash \Suc(n) * 0 = n * 0$ by rewriting 2 with nat_plus_def_1.
4. $n * 0 = 0 \vdash \Suc(n) * 0 = 0$ by rewriting 3 with 1.
5. $\vdash n * 0 = 0 \to \Suc(n) * 0 = 0$ implies_intr $n * 0 = 0$ from 4.
6. $\vdash \forall n.\,n * 0 = 0 \to \Suc(n) * 0 = 0$ by forall_intr $n$ from 5.
7. $n * 0 = 0$ by apply_theorem nat_induct from 0, 6.

This is realized in Python as follows:

In [10]:
n = Var('n', NatType)

pt0 = refl(Nat(0) * 0).on_rhs(rewr_conv('nat_times_def_1'))
pt1 = ProofTerm.assume(Eq(n * 0, 0))
pt2 = refl(Suc(n) * 0).on_rhs(rewr_conv('nat_times_def_2'))
pt3 = pt2.on_rhs(rewr_conv('nat_plus_def_1'))
pt4 = pt3.on_rhs(rewr_conv(pt1))
pt5 = pt4.implies_intr(Eq(n * 0, 0))
pt6 = pt5.forall_intr(n)
pt7 = apply_theorem('nat_induct', pt0, pt6, inst=Inst(P=Lambda(n, Eq(n * 0, 0)), x=n))
print(pt2)

ProofTerm(⊢ Suc n * 0 = 0 + n * 0)


The same approach can be used to prove other properties of addition and multiplication, such as associativity, commutativity, and distributivity. We leave the proofs to later, when the use of tactics make them more convenient.

## Binary representation

Representing natural numbers using $\Suc$ is straightforward but far from efficient. It is exponentially more expensive than decimal or binary notation. The same holds for carrying out arithmetic operations on natural numbers. Hence, in practice we use binary notation to represent natural numbers. A *binary number* is a term formed using $\zero$, $\one$, and two functions $\bitz$ and $\bito$. The two functions are defined as follows:

In [11]:
theory.print_theorem('bit0_def', 'bit1_def')

bit0_def: ⊢ bit0 n = n + n
bit1_def: ⊢ bit1 n = n + n + 1


It can be seen that each nonzero natural number can be represented uniquely as a sequence of applications of $\bitz$ and $\bito$ to $\one$. The sequence of applications corresponds to the reverse of the binary representation of that number. For example, the number $10$ is $1010$ in binary form, and we have:

$$ 10 = \bitz(\bito(\bitz(\one))) $$

In holpy, the natural numbers zero and one are represented directly as constants. Natural numbers greater than one are represented using the function $\hbox{of_nat} :: nat \To nat$ applied to a natural number. The use of $\hbox{of_nat}$ is for consistency with other numeral types, such as integers and real numbers. For natural numbers it is just the identity function. The pretty printer automatically prints a natural number in this form as numerals. However, we can deconstruct the function and argument to see its detailed structure:

In [12]:
print(Nat(10))
print(Nat(10).fun)
print(Nat(10).arg)

(10::nat)
(of_nat::nat ⇒ nat)
bit0 (bit1 (bit0 1))


Another way to see the detailed binary form is to use the `print_basic` method for terms:

In [13]:
print(Nat(10).print_basic())

of_nat (bit0 (bit1 (bit0 one)))


Addition on binary numbers is carried using several lemmas about $\bitz$ and $\bito$:

In [14]:
theory.print_theorem('bit0_bit0_add', 'bit0_bit1_add', 'bit1_bit0_add', 'bit1_bit1_add')

bit0_bit0_add: ⊢ bit0 m + bit0 n = bit0 (m + n)
bit0_bit1_add: ⊢ bit0 m + bit1 n = bit1 (m + n)
bit1_bit0_add: ⊢ bit1 m + bit0 n = bit1 (m + n)
bit1_bit1_add: ⊢ bit1 m + bit1 n = bit0 (Suc (m + n))


It reduces addition of two binary numbers to addition with smaller cases, as well as evaluation of $\Suc$ on a binary number, for which we also have:

In [15]:
theory.print_theorem('one_Suc', 'bit0_Suc', 'bit1_Suc')

one_Suc: ⊢ Suc 1 = bit0 1
bit0_Suc: ⊢ Suc (bit0 n) = bit1 n
bit1_Suc: ⊢ Suc (bit1 n) = bit0 (Suc n)


Hence, we can implement evaluation of $\Suc$ on a binary number as follows:

In [16]:
class Suc_conv(Conv):
    def get_proof_term(self, t):
        return refl(t).on_rhs(top_conv(('nat_one_def', 'sym'), 'one_Suc', 'bit0_Suc', 'bit1_Suc'))

In [17]:
print(refl(Suc(Binary(10))).on_rhs(Suc_conv()))
print(refl(Suc(Nat(0))).on_rhs(Suc_conv()))
print(refl(Suc(Binary(3))).on_rhs(Suc_conv()))

ProofTerm(⊢ Suc (bit0 (bit1 (bit0 1))) = bit1 (bit1 (bit0 1)))
ProofTerm(⊢ Suc 0 = 1)
ProofTerm(⊢ Suc (bit1 1) = bit0 (bit0 1))


Next, evaluation of addition. Here a `repeat_conv` is needed because when using `bit1_bit1_add`, the result requires first evaluating the subterm $m+n$, then evaluating $\Suc$ on the result, which does not fit with the top-down order. Hence `top_conv` is not able to finish evaluation in one run.

In [18]:
class add_conv(Conv):
    def get_proof_term(self, t):
        return refl(t).on_rhs(repeat_conv(top_conv(
            'nat_plus_def_1', 'add_0_right', 'add_1_left', 'add_1_right',
            'bit0_bit0_add', 'bit0_bit1_add', 'bit1_bit0_add', 'bit1_bit1_add', Suc_conv()
        )))

In [19]:
print(refl(Binary(10)+Binary(6)).on_rhs(add_conv()))

ProofTerm(⊢ bit0 (bit1 (bit0 1)) + bit0 (bit1 1) = bit0 (bit0 (bit0 (bit0 1))))


Likewise the theorems used for evaluating multiplication is as follows:

In [20]:
theory.print_theorem('bit0_bit0_mult', 'bit0_bit1_mult', 'bit1_bit0_mult', 'bit1_bit1_mult')

bit0_bit0_mult: ⊢ bit0 m * bit0 n = bit0 (bit0 (m * n))
bit0_bit1_mult: ⊢ bit0 m * bit1 n = bit0 (m * bit1 n)
bit1_bit0_mult: ⊢ bit1 m * bit0 n = bit0 (bit1 m * n)
bit1_bit1_mult: ⊢ bit1 m * bit1 n = bit1 (m + n + bit0 (m * n))


And the conversion can be implemented as:

In [21]:
class mult_conv(Conv):
    def get_proof_term(self, t):
        return refl(t).on_rhs(repeat_conv(top_conv(
            'nat_times_def_1', 'mult_0_right', 'mult_1_left', 'mult_1_right',
            'bit0_bit0_mult', 'bit0_bit1_mult', 'bit1_bit0_mult', 'bit1_bit1_mult', add_conv()
        )))

In [22]:
print(refl(Binary(10)*Binary(6)).on_rhs(mult_conv()))

ProofTerm(⊢ bit0 (bit1 (bit0 1)) * bit0 (bit1 1) = bit0 (bit0 (bit1 (bit1 (bit1 1)))))
